In [1]:
import numpy as np
import numba
import cython

%load_ext cython

Генерируем случайную матрицу $1000 \times 1000$

In [2]:
matr = np.random.random(size=(1000, 1000))

## Чистый Python

In [3]:
def func_python(a):
    s = 0
    for y in range(1, a.shape[0]):
        for x in range(1, a.shape[1]):
            s += a[y, x] * a[y, x-1] + a[y-1, x-1] 
            
    return s

## Numba

JIT-компилятор, умеет транлировать `Python` (в основном `NumPy`-ориентированный) в машинный код для выполнения на CPU/GPU.

In [4]:
func_numba = numba.jit(func_python)

## Cython

Специализированный язык для написания нативных расширений для `Python` (`Cython` -> `C` -> `Python Module`)

In [5]:
%%cython

cimport numpy 

def func_cython(numpy.ndarray['double', ndim=2] a):
    cdef int x
    cdef int y 
    cdef double s = 0.0
    
    for y in range(1, a.shape[0]):
        for x in range(1, a.shape[1]):
            s += a[y, x] * a[y, x-1] + a[y-1, x-1]
    
    return s

## Результаты

In [6]:
%%timeit
func_python(matr)

862 ms ± 34.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
func_numba(matr)

1.19 ms ± 57.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%%timeit
func_cython(matr)

2.19 ms ± 19.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# JAX

[`JAX`](https://github.com/google/jax) - `NumPy`-совместимая библиотека с jit-компилятором для GPU/TPU и возможностью автоматического дифференцирования.

In [9]:
import jax
import jax.numpy as np

Массивы создаются аналогично `NumPy`

In [11]:
np.array([1, 2, 4, 5, 6])

DeviceArray([1, 2, 4, 5, 6], dtype=int32)

Опишем функцию:

$f(x, y) = x^2 + 5xy + 4$

In [12]:
def func(x, y):
    return x**2 + 5*x*y + 4.

jitted_func = jax.jit(func)
arr = np.arange(1., 1000., 1.)

Для копиляции используется `jax.jit`

In [13]:
jitted_func = jax.jit(func)

Измерим производительность

In [14]:
arr = np.arange(1., 1000., 1.)

%timeit jitted_func(arr, arr)

%timeit func(arr, arr)

113 µs ± 2.13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
914 µs ± 119 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Для подсчета градиента использутся `jax.grad`.

$$f(x, y) = x^2 + 5xy + 4$$

$$\frac{\partial{f}}{\partial{x}} = 2x + 5y $$

$$\frac{\partial{f}}{\partial{y}} = 5x $$

$$\nabla{f} = (2x + 5y, 5x)$$

$$\nabla{f(5, 10)} = (60, 25)$$

In [15]:
grad = jax.jit(jax.grad(func, argnums=(0, 1)))
grad(5., 10.)

(DeviceArray(60., dtype=float32), DeviceArray(25., dtype=float32))

Удобнее задавать все аргументы в одном массиве

In [16]:
def func(x):    
    return x[0] ** 2 +  x[1] ** 2 + x[2] ** 2
    ## return np.sum(x ** 2)

grad = jax.jit(jax.grad(func, argnums=0))

In [17]:
func(np.array([1., 3., 4.]))

DeviceArray(26., dtype=float32)

In [18]:
grad(np.array([1., 4., 5.]))

DeviceArray([ 2.,  8., 10.], dtype=float32)